In [1]:
import numpy as np
from scipy import optimize
from capsolpy import defcapsol as cs
# reload(capsolpy)

ImportError: libifport.so.5: cannot open shared object file: No such file or directory

# Develop CapSolPy

- `cs.generategrid` currently works
- `cs.setupprobe` does not (seg faults on MAC OS X, even though it works in fortran)
- `cs.setupprobe` is relatively simple - we could translate that code to python

In [3]:
# Parameters
nuni = 1
n = 500
m = 500
h0 = 0.5
rho_max = 1e6
z_max = 1e6
z = 10.0
l_js = 20
l = l_js + int(np.round(z/h0))
verbose=0
hsam = 10.0
rtip = 20.0
theta = 15 * 180 / np.pi 
hcone = 15000.0
rcant = 500.0
dcant = 15000.0
eps_r = 3.0


# r, hn, hm, zm = cs.generategrid(nuni, h0, rho_max,z_max,z,hsam,n,m,l,verbose)

In [ ]:
# print(cs.generategrid.__doc__)

In [39]:
def geometric_sum(r, a1, n):
    return sum(np.array([a1 * r ** i for i in range(1, n + 1)]))

def find_ratio(total, h0, terms):
    r = 1.00001
    s = 0
    while s < total:
        s = geometric_sum(r, h0, terms)
        r += 0.00001
    return r

def guni_grid(nuni, n, h0, grid_max):
    """Generates geometric grid which can be used for the radial and z directions. 
    
    nuni: number of uniform grid points 
    n: number of gird points
    h0: initial grid spacing
    grid_max: rho_max or z_max, maximum grid size"""
    
    
    r = np.zeros(n+1)
    dr = np.zeros(n) # Could also use n+1
    dr[:nuni] = h0
    r[1:nuni+1] = np.cumsum(dr[:nuni])
    r_left = grid_max-(nuni*h0)
    n_left=n-nuni
    ratio= find_ratio(r_left, h0, n_left)
    for i in range(n_left):
        dr[nuni+i]= h0*ratio**(i+1)
    r[1:] = np.cumsum(dr[:])
    return r, dr

- `r` is the radii (`n` + 1 points spaced from 0 to `rho_max`)
- `hn` is the spacing between radii

In [40]:
guni_grid(5,10,.5,10)

(array([ 0.        ,  0.5       ,  1.        ,  1.5       ,  2.        ,
         2.5       ,  3.19591   ,  4.16449146,  5.5125825 ,  7.38888257,
        10.00035454]),
 array([0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.69591   , 0.96858146, 1.34809104, 1.87630007, 2.61147197]))

In [41]:
find_ratio(7.5, 0.5, 5)

1.391820000002567

In [55]:
# Parameters
nuni = 50 #number of uniformly spaced grid points
n = 500 # number of radial grid points
m = 500 # number of z+ grid points
h0 = 0.5 # initial grid spacing
rho_max = 1e6 # maximum radial size
z_max = 1e6 # maximum height size
z = 10.0 # tip-sample separation
l_js = 20  #m-, grid points in sample
l = l_js + int(np.round(z/h0)) # actual number of negative z points
verbose=0 
hsam = 15.0 #sample thickness
rtip = 20.0 #tip radius
theta = 15 * 180 / np.pi #hallf angle of tip
hcone = 15000.0 # height of cone
rcant = 15000.0 # cantilever radius
dcant = 3000.0 #cantilever thickness
eps_r = 3.0 #dielectric constant 
 

# r, hn, hm, zm = cs.generategrid(nuni, h0, rho_max,z_max,z,hsam,n,m,l,verbose)

In [57]:
r,dr=guni_grid(nuni, n, h0, rho_max)
z_plus, dz_plus=guni_grid(nuni, m, h0, z_max)
print(r[-5:] , dr[5: -5])

[ 913232.53641719  935350.93505936  958005.04131665  981207.83002749
 1004972.59028091] [5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.12110000e-01 5.24513304e-01 5.37217016e-01
 5.50228413e-01 5.63554945e-01 5.77204245e-01 5.91184132e-01
 6.05502612e-01 6.20167885e-01 6.35188351e-01 6.50572613e-01
 6.66329482e-01 6.82467982e-01 6.98997357e-01 7.15927073e-

In [58]:
np.sum(dr)

1004972.5902809092

In [59]:
print(z_plus[-5:], dz_plus[5:-5])

[ 913232.53641719  935350.93505936  958005.04131665  981207.83002749
 1004972.59028091] [5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.00000000e-01 5.00000000e-01 5.00000000e-01
 5.00000000e-01 5.12110000e-01 5.24513304e-01 5.37217016e-01
 5.50228413e-01 5.63554945e-01 5.77204245e-01 5.91184132e-01
 6.05502612e-01 6.20167885e-01 6.35188351e-01 6.50572613e-01
 6.66329482e-01 6.82467982e-01 6.98997357e-01 7.15927073e-

In [90]:
def gapsam_gridsize(h0, hsam, z):
    n_sample=int(np.round(hsam/h0))
    n_gap=int(np.round(z/h0))
    gapsam=n_sample + n_gap + 1 
    return gapsam
def generate_gapsam_grid(h0, hsam, z):
    N=gapsam_grid(h0, hsam, z)
    z_minus=np.arange(N)*h0 + -(z+hsam) + -h0
    return z_minus

In [91]:
z_minus=generate_gapsam_grid(h0, hsam, z)

In [92]:
z_grid=np.r_[z_minus, z_plus]

In [93]:
dz_minus=np.arange(N)*h0 

In [94]:
z_grid

array([-2.55000000e+01, -2.50000000e+01, -2.45000000e+01, -2.40000000e+01,
       -2.35000000e+01, -2.30000000e+01, -2.25000000e+01, -2.20000000e+01,
       -2.15000000e+01, -2.10000000e+01, -2.05000000e+01, -2.00000000e+01,
       -1.95000000e+01, -1.90000000e+01, -1.85000000e+01, -1.80000000e+01,
       -1.75000000e+01, -1.70000000e+01, -1.65000000e+01, -1.60000000e+01,
       -1.55000000e+01, -1.50000000e+01, -1.45000000e+01, -1.40000000e+01,
       -1.35000000e+01, -1.30000000e+01, -1.25000000e+01, -1.20000000e+01,
       -1.15000000e+01, -1.10000000e+01, -1.05000000e+01, -1.00000000e+01,
       -9.50000000e+00, -9.00000000e+00, -8.50000000e+00, -8.00000000e+00,
       -7.50000000e+00, -7.00000000e+00, -6.50000000e+00, -6.00000000e+00,
       -5.50000000e+00, -5.00000000e+00, -4.50000000e+00, -4.00000000e+00,
       -3.50000000e+00, -3.00000000e+00, -2.50000000e+00, -2.00000000e+00,
       -1.50000000e+00, -1.00000000e+00, -5.00000000e-01,  0.00000000e+00,
        5.00000000e-01,  

In [97]:
R,Z=np.meshgrid(r, z_grid)

In [98]:
R,Z

(array([[0.00000000e+00, 5.00000000e-01, 1.00000000e+00, ...,
         9.58005041e+05, 9.81207830e+05, 1.00497259e+06],
        [0.00000000e+00, 5.00000000e-01, 1.00000000e+00, ...,
         9.58005041e+05, 9.81207830e+05, 1.00497259e+06],
        [0.00000000e+00, 5.00000000e-01, 1.00000000e+00, ...,
         9.58005041e+05, 9.81207830e+05, 1.00497259e+06],
        ...,
        [0.00000000e+00, 5.00000000e-01, 1.00000000e+00, ...,
         9.58005041e+05, 9.81207830e+05, 1.00497259e+06],
        [0.00000000e+00, 5.00000000e-01, 1.00000000e+00, ...,
         9.58005041e+05, 9.81207830e+05, 1.00497259e+06],
        [0.00000000e+00, 5.00000000e-01, 1.00000000e+00, ...,
         9.58005041e+05, 9.81207830e+05, 1.00497259e+06]]),
 array([[-2.55000000e+01, -2.55000000e+01, -2.55000000e+01, ...,
         -2.55000000e+01, -2.55000000e+01, -2.55000000e+01],
        [-2.50000000e+01, -2.50000000e+01, -2.50000000e+01, ...,
         -2.50000000e+01, -2.50000000e+01, -2.50000000e+01],
        [-2.4